In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from enum import Enum


class Sentiment(BaseModel):
    """
    A model to represent sentiment analysis results.
    """
    sentiment: str = Field(
        description="The sentiment of the text, e.g., 'positive', 'negative', or 'neutral'."
    )
    language: str = Field(
        description="The language of the text, e.g., 'en' for English, 'es' for Spanish, 'pt-br' for Portuguese (Brazil)."
    )
    confidence: float = Field(
        description="The confidence score of the sentiment analysis, between 0 and 1."
    )

tool_sentiment_analysis = convert_to_openai_function(Sentiment)

text = "I play football with my friends every weekend. It's a lot of fun!"

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that performs sentiment analysis."),
        ("user", "Analyze the sentiment of the following text: {text}"),
    ]
)

chain = prompt | chat.bind(functions=[tool_sentiment_analysis], function_call="auto")
# chain.invoke({"text": text})


# Categorização de Texto:
solicitacoes = [
    "Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?",
    "Não consigo acessar minha conta. A senha não está funcionando!",
    "Estou tendo dificuldades para instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar meu endereço de entrega na plataforma.",
    "O meu laptop não está ligando. Acho que é um problema no cabo de energia.",
    "Não consigo ver meus arquivos no aplicativo. Existe alguma solução?"
]

class SetorEnum(str, Enum):
    """
    Enum to categorize text into different sectors.
    """
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros"

class DirecionaSuporte(BaseModel):
    """
    A model to categorize text into different support sectors.
    """
    setor: SetorEnum = Field(
        description="The sector to which the request should be directed, e.g., 'suporte_hardware', 'suporte_software', 'suporte_conta', or 'outros'."
    )
    confidence: float = Field(
        description="The confidence score of the categorization, between 0 and 1."
    )


tool_direciona_suporte = convert_to_openai_function(DirecionaSuporte)
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Criar o prompt com as instruções detalhadas para o modelo
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou 
outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, 
etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, 
devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chain = prompt | chat.bind(functions=[tool_direciona_suporte], function_call="auto") | JsonOutputFunctionsParser()
for solicitacao in solicitacoes:
    response = chain.invoke({"input": solicitacao})
    print(f"Solicitação: {solicitacao}\nResposta: {response}\n")
